In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
def gatherData(url):
    url = url
    html = requests.get(
                url,
                allow_redirects=False,
                headers={"User-Agent": "waybackpack"},
                stream=True
            )

    soup = BeautifulSoup(html.text, 'html')
    totalItems = soup.find_all(class_ = 'state-wrap')

    df = pd.DataFrame(columns=["state","area","population","date","type"])

    for item in totalItems: 

        if item.find_all(class_ ="l-place"):
            state = item.h3.text

            for place in item.find_all(class_ = "place-wrap"):

                date = place.find(class_ ="l-date").extract().text
                population = place.find(class_ ="l-population").extract().text
                area = place.find(class_ ="l-place").text
                type = place.find(class_ ="l-order").text

                dict = {'state': state, 'area': area, 'population': population,'date':date,'type':type}

                df = df.append(dict, ignore_index=True)
        else:
            date = item.find(class_ ="l-date").extract().text
            population = item.find(class_ ="l-population").extract().text
            area = "ALL"
            type = item.find(class_ ="l-order").text
            state = item.h3.text

            dict = {'state': state, 'area': area, 'population': population,'date':date,'type':type}
            df = df.append(dict, ignore_index=True)

    # Reformat some of the columns
    df['population'] = df['population'].str.replace("About ","")
    df['date'] = df['date'].str.replace(", effective ","")
    df['state'] = df['state'].str.strip()
    df['area'] = df['area'].str.strip()

    # Make date a date-time column
    conditions = [
    (df['date'].str.contains('..-')),
    (df['date'].str.contains('^.-',regex=True)),
    (df['date'].str.contains('March ')),
    (df['date'].str.contains('April '))]
    choices = ["2020-03-" + df['date'].str[0:2], "2020-04-0"+df["date"].str[0], "2020-03-" + df["date"].str[6:8],"2020-04-0" + df["date"].str[6:7]]
    df['date'] = np.select(conditions, choices)
    
    return df

In [3]:
df_0324 = gatherData("https://web.archive.org/web/20200324041533/https://www.nytimes.com/interactive/2020/us/coronavirus-stay-at-home-order.html")
df_0325 = gatherData("https://web.archive.org/web/20200325184603/https://www.nytimes.com/interactive/2020/us/coronavirus-stay-at-home-order.html")
df_0326 = gatherData("https://web.archive.org/web/20200326002332/https://www.nytimes.com/interactive/2020/us/coronavirus-stay-at-home-order.html")
df_0327 = gatherData("https://web.archive.org/web/20200327003315/https://www.nytimes.com/interactive/2020/us/coronavirus-stay-at-home-order.html")
df_0328 = gatherData("https://web.archive.org/web/20200328020115/https://www.nytimes.com/interactive/2020/us/coronavirus-stay-at-home-order.html")
df_0329 = gatherData("https://web.archive.org/web/20200329000754/https://www.nytimes.com/interactive/2020/us/coronavirus-stay-at-home-order.html")
df_0331 = gatherData("https://web.archive.org/web/20200331015116/https://www.nytimes.com/interactive/2020/us/coronavirus-stay-at-home-order.html")
df_0402 = gatherData("https://web.archive.org/web/20200402134943/https://www.nytimes.com/interactive/2020/us/coronavirus-stay-at-home-order.html")
df_0403 = gatherData("https://web.archive.org/web/20200403221428/https://www.nytimes.com/interactive/2020/us/coronavirus-stay-at-home-order.html")
df_0407 = gatherData("https://web.archive.org/web/20200407140353/https://www.nytimes.com/interactive/2020/us/coronavirus-stay-at-home-order.html")
df_0420 = gatherData("https://web.archive.org/web/20200421023813/https://www.nytimes.com/interactive/2020/us/coronavirus-stay-at-home-order.html")
df_0423 = gatherData("https://www.nytimes.com/interactive/2020/us/coronavirus-stay-at-home-order.html")

In [4]:
df_final = pd.concat([df_0324,df_0325,df_0326,df_0327,df_0328,df_0329,df_0331,df_0402,df_0403,df_0407,df_0420,df_0423])
df_final.drop_duplicates().query('date !=0').to_csv("shelterinplace.csv", index=False)